In [1]:
!ls /kaggle/input/sccos-dota/sccos_dota

test  train  val


In [2]:
!pip install wandb -q
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_tanish")

wandb.login(key = secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanish-jain140301 (tanish1403) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!python --version

Python 3.11.13


In [4]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 -q
# !pip install mmcv-full==1.7.1 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html -q
!pip install mmdet==2.28.2 -q
!pip install -U openmim -q
!mim install "mmengine>=0.7.0" -q
!pip install xmltodict -q  # For dataset conversion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 393.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.2 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.5 MB/s eta 0:00:00
    

In [5]:
!mim install 'mmcv==2.0.1'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 8.1 MB/s eta 0:00:00


In [6]:
!mim install 'mmcv-full'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 9.7 MB/s eta 0:00:00


In [7]:
# Step 6: Prepare Dataset (SCCOS to DOTA format)
import os
import shutil
import random
import xmltodict
import tqdm
import math

dataset_path = "/kaggle/input/sccos-dataset/"
working_dir = "/kaggle/working/sccos_dota"
train_images_dir = os.path.join(working_dir, "train/images")
train_labels_dir = os.path.join(working_dir, "train/labels")
val_images_dir = os.path.join(working_dir, "val/images")
val_labels_dir = os.path.join(working_dir, "val/labels")
test_images_dir = os.path.join(working_dir, "test/images")
test_labels_dir = os.path.join(working_dir, "test/labels")

In [8]:
# Step 4: Clone and Install MMRotate 0.3.4
!git clone https://github.com/zhangpeng2001/nirnet.git /kaggle/working/mmrotate
%cd /kaggle/working/mmrotate
# !git checkout v0.3.4  # Ensure exact version
!pip install -r requirements/build.txt -q
!pip install -v -e . -q

Cloning into '/kaggle/working/mmrotate'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 482 (delta 143), reused 474 (delta 135), pack-reused 0 (from 0)
Receiving objects: 100% (482/482), 11.52 MiB | 31.29 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/kaggle/working/mmrotate
Obtaining file:///kaggle/working/mmrotate
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 5.0 MB/s eta 0:00:00
  Running setup.py develop for mmrotate


In [9]:
# Step 5: Verify Installations
!pip list | grep -E 'torch|mmcv|mmdet|mmengine|mmrotate'

mmcv                                  2.0.1
mmcv-full                             1.7.2
mmdet                                 2.28.2
mmengine                              0.10.7
mmrotate                              0.3.4               /kaggle/working/mmrotate
pytorch-ignite                        0.5.2
pytorch-lightning                     2.5.2
torch                                 2.0.1+cu118
torchao                               0.10.0
torchaudio                            2.0.2+cu118
torchdata                             0.11.0
torchinfo                             1.8.0
torchmetrics                          1.7.3
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.15.2+cu118


In [10]:
%%writefile configs/oriented_reppoints/oriented_reppoints_r50_fpn_1x_sccos.py
_base_ = [
    '../_base_/datasets/dotav1.py', '../_base_/schedules/schedule_1x.py',
    '../_base_/default_runtime.py'
]

angle_version = 'le135'
norm_cfg = dict(type='GN', num_groups=32, requires_grad=True)
model = dict(
    type='RotatedRepPoints',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        zero_init_residual=False,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=1,
        add_extra_convs='on_input',
        num_outs=5,
        norm_cfg=norm_cfg),
    bbox_head=dict(
        type='OrientedRepPointsHead',
        num_classes=1,
        in_channels=256,
        feat_channels=256,
        point_feat_channels=256,
        stacked_convs=3,
        num_points=9,
        gradient_mul=0.3,
        point_strides=[8, 16, 32, 64, 128],
        point_base_scale=2,
        norm_cfg=norm_cfg,
        loss_cls=dict(
            type='FocalLoss',
            use_sigmoid=True,
            gamma=2.0,
            alpha=0.25,
            loss_weight=1.0),
        loss_bbox_init=dict(type='ConvexGIoULoss', loss_weight=0.375),
        loss_bbox_refine=dict(type='ConvexGIoULoss', loss_weight=1.0),
        loss_spatial_init=dict(type='SpatialBorderLoss', loss_weight=0.05),
        loss_spatial_refine=dict(type='SpatialBorderLoss', loss_weight=0.1),
        init_qua_weight=0.2,
        top_ratio=0.4),
    # training and testing settings
    train_cfg=dict(
        init=dict(
            assigner=dict(type='ConvexAssigner', scale=4, pos_num=1),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        refine=dict(
            assigner=dict(
                type='MaxConvexIoUAssigner',
                pos_iou_thr=0.1,
                neg_iou_thr=0.1,
                min_pos_iou=0,
                ignore_iof_thr=-1),
            allowed_border=-1,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        nms_pre=2000,
        min_bbox_size=0,
        score_thr=0.05,
        nms=dict(iou_thr=0.4),
        max_per_img=2000))

dataset_type = 'DOTADataset'
# data_root = '/kaggle/working/sccos_dota/'
data_root = '/kaggle/input/sccos-dota/sccos_dota/'
classes = ('ship',)  # Explicitly define classes
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024)),
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le135'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='RResize'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]
data = dict(
    samples_per_gpu=4,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train/labels/',
        img_prefix=data_root + 'train/images/',
        pipeline=train_pipeline,
        version='le135',
        classes=('ship',)),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'val/labels/',
        img_prefix=data_root + 'val/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test/labels/',
        img_prefix=data_root + 'test/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)))
evaluation = dict(interval=1, metric='mAP', save_best='mAP') # Add save_best here
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=1000,
    warmup_ratio=0.3333333333333333,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=25)
checkpoint_config = dict(interval=1) # Keep interval for saving
log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook',
             init_kwargs=dict(
                 project='SOTA',
                 name='ORCNN_train'),
             interval=100)
    ])

Writing configs/oriented_reppoints/oriented_reppoints_r50_fpn_1x_sccos.py


In [11]:
%cd /kaggle/working/mmrotate
!python tools/analysis_tools/get_flops.py configs/oriented_reppoints/oriented_reppoints_r50_fpn_1x_sccos.py

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
RotatedRepPoints(
  35.424 M, 96.777% Params, 194.241 GFLOPs, 100.000% FLOPs, 
  (backbone): ResNet(
    23.283 M, 63.608% Params, 86.095 GFLOPs, 44.324% FLOPs, 
    (conv1): Conv2d(0.0 M, 0.000% Params, 2.466 GFLOPs, 1.270% FLOPs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(0.0 M, 0.000% Params, 0.034 GFLOPs, 0.017% FLOPs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(0.0 M, 0.000% Params, 0.017 GFLOPs, 0.009% FLOPs, inplace=True)
    (maxpool): MaxPool2d(0.0

In [12]:
# Step 8: Train the Model
%cd /kaggle/working/mmrotate
!mkdir -p /kaggle/working/runs/ORepp_train


!python tools/train.py \
    configs/configs/oriented_reppoints/oriented_reppoints_r50_fpn_1x_sccos.py \
    --work-dir /kaggle/working/runs/ORepp_train \
    --gpus 1 \


print("Training completed.")

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
Traceback (most recent call last):
  File "/kaggle/working/mmrotate/tools/train.py", line 194, in <module>
    main()
  File "/kaggle/working/mmrotate/tools/train.py", line 86, in main
    cfg = Config.fromfile(args.config)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mmcv/utils/config.py", line 342, in fromfile
    cfg_dict, cfg_text = Config._file2dict(filename,
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/mmcv/utils/config.py", line 185, in _file2

In [13]:
# Test FIRNet (uncomment to run)
import os
import glob

%cd /kaggle/working/mmrotate

# Step 1: Find the best checkpoint file automatically
checkpoint_dir = '/kaggle/working/runs/ORepp_train'
best_checkpoint_path = None
# Search for the best checkpoint file
for filename in glob.glob(os.path.join(checkpoint_dir, 'best_mAP_*.pth')):
    best_checkpoint_path = filename
    break  # Take the first one found

if best_checkpoint_path:
    print(f"Found best checkpoint: {best_checkpoint_path}")
    
    # Step 2: Run the test command with the found checkpoint
    !mkdir -p /kaggle/working/runs/ORepp_test
    !python tools/test.py \
        configs/oriented_reppoints/oriented_reppoints_r50_fpn_1x_sccos.py \
        {best_checkpoint_path} \
        --eval mAP \
        --out /kaggle/working/runs/ORepp_test/results.pkl \
        --show-dir /kaggle/working/runs/ORepp_test/vis
    
    print("FIRNet testing completed. Results saved.")
else:
    print(f"Error: No best checkpoint found in {checkpoint_dir}. Please ensure training was successful.")

/kaggle/working/mmrotate
Error: No best checkpoint found in /kaggle/working/runs/ORepp_train. Please ensure training was successful.
